<span id="menu"></span>
# Zipline order_percent
* [Order_percent](#Order_percent)
* [Order_target_percent](#Order_target_percent)

order() 和 order_percent() 的差別是，前者由使用者指定股數，後者是指定投資組合總值 (portfolio value) 的一個比例。

他的parameters是:

- asset: 股票
- percent: 介於-1和1，正long負short
- limit_price: 限價，預設為None
- stop_price: 指損價，預設為None
- style: Execution style，預設為None，也可以做limit/stop的功能。如果limit_price或stop_price已經填了，就不用也不能填這裡
    
這函數的大致運作，是先用輸入的比例還有目前總值，算出要買的金額，再搭配最近的股價，算出要買的股數，最後代入order函數。

order函數中，若股數和最近整數相差不到0.0001，就取最近整數，否則直接去掉小數 (5.5 -> 5.0, -5.5 -> -5.0)。

# 範例講解

## 設定環境

In [2]:
import pandas as pd
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key-------------------------------------------
tejapi.ApiConfig.api_key = 'your key'  
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"
os.environ['TEJAPI_KEY'] = 'your key' 
# date----------------------------------------------
# set date
start='2018-07-24'
end='2018-08-14'
os.environ['mdate'] = '20180724 20180814'      # start+' '+end #'20221011 20221223'

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)
# calendar------------------------------------------
calendar_name='TEJ'  # US equities  XTAI

# bundle_name---------------------------------------
bundle_name = 'tquant'

from zipline.utils.calendar_utils import get_calendar
if get_calendar(calendar_name).is_session(start_dt)==False:
    start_dt=get_calendar(calendar_name).next_open(start_dt)
    
if get_calendar(calendar_name).is_session(end_dt)==False:
    end_dt=get_calendar(calendar_name).previous_close(pd.Timestamp(end_dt))
    
from zipline.api import    *

from zipline import run_algorithm  
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from zipline.utils.run_algo import  (get_transaction_detail,
                                     get_record_vars)

from zipline.sources.TEJ_Api_Data import (get_Treasury_Return,
                                          get_Benchmark_Return,)

import numpy as np
import pandas as pd
from zipline.utils import run_algo
from zipline.data import bundles


coid='1101 2330 IR0001 5844'
    
os.environ['ticker'] = coid       #'1101 1102'   #coid

!zipline ingest -b tquant

Merging daily equity files:


[2023-08-08 09:30:14.541438] INFO: zipline.data.bundles.core: Ingesting tquant.


<span id="Order_percent"></span>
## Order_percent介紹
[Return to Menu](#menu)

## 投資1101、2330兩支股票

### 設置交易策略
#### def handle_data(context, data):

    在交易策略的第一個交易時間點（i 等於 0）時
    if context.i == 0:
        order_percent(symbol('1101'), 0.3, limit_price = 43.5)
        使用限價訂單購買股票 '1101'，量為投資組合當前價值的 30%，限價為 43.5
        order_percent(symbol('2330'), 0.3, limit_price = 240.6)
        使用限價訂單購買股票 '2330'，量為投資組合當前價值的 30%，限價為 240.6
    
    在交易策略的第一個交易時間點（i 等於 6）時
    if context.i == 6:
        
        order_percent(symbol('1101'), 0.2)
        購買股票 '1101'，量為投資組合當前價值的 20%
        order_percent(symbol('2330'), 0.2)
        購買股票 '2330'，量為投資組合當前價值的 20%
    
    記錄投資組合中所有資產的收盤價
    record(close=data.current(context.asset, 'close'))
    context.i += 1

In [3]:
def initialize(context):
    context.i = 0
    context.tickers = ['1101','2330']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order_percent(symbol('1101'), 0.3, limit_price = 43.5)
        order_percent(symbol('2330'), 0.3, limit_price = 240.6)
        
    if context.i == 6:
        order_percent(symbol('1101'), 0.2)
        order_percent(symbol('2330'), 0.2)

    record(close=data.current(context.asset, 'close'))
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e5
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')


In [6]:
closing_price = tejapi.get('TWN/APIPRCD',
                           coid=['1101','2330'], 
                           opts={'columns':['mdate','coid','close_d']}, 
                           mdate={'gte':start_dt,'lte':end_dt }, 
                           paginate=True)

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

## 回測結果

#### 收盤價

In [7]:
closing_price

,mdate,coid,close_d
None,,,
0,2018-07-24,1101,45.50
1,2018-07-25,1101,45.10
2,2018-07-26,1101,40.50
3,2018-07-27,1101,40.30
4,2018-07-30,1101,40.70
5,2018-07-31,1101,39.35
6,2018-08-01,1101,41.05
7,2018-08-02,1101,40.60
8,2018-08-03,1101,40.45


### 下單紀錄

In [8]:
transactions

amount  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330       124   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101       725   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101       497   
                          Equity(1 [2330]) 2330        82   

                                                                         dt  \
date                      asset            symbol                             
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330   2018-07-25 13:30:00+08:00   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-31 13:30:00+08:00   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-02 13:30:00+08:00   
                          Equity(1 [2330]) 2330   2018-08-02 13:30:00+08:00   

                                                    price  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330    240.50   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101     39.35   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101     40.60   
                          Equity(1 [2330]) 2330    244.50   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330    a4a72ad3229541c9b655745f305c7b0d   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101    d4f04b538af140298289d1cc96620a1c   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101    6121b364417b471f92de68d5c246237f   
                          Equity(1 [2330]) 2330    00099a5ddcfb496db3b6aea457670fd3   

                                                  commission  
date                      asset            symbol             
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330         None  
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101         None  
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101         None  
                          Equity(1 [2330]) 2330         None

## 1

第一天 (7/24) 建立兩單，1101、2330各30%，limit_price分別為43.5, 240.6。因為當天1101收盤價是45.5

#### 在7/24時起始現金為100,000，30%的金額為30000
#### 1101購買股數 = 30000 / 45.5 = 659 股
#### 2330購買股數 = 30000 / 241 = 124股

In [9]:
performance['orders'][0]

[{'id': 'd4f04b538af140298289d1cc96620a1c',
  'dt': Timestamp('2018-07-24 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-24 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 659,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': 43.5,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.OPEN: 0>},
 {'id': 'a4a72ad3229541c9b655745f305c7b0d',
  'dt': Timestamp('2018-07-24 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-24 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 124,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': 240.6,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2330]),
  'status': <ORDER_STATUS.OPEN: 0>}]

In [10]:
closing_price[closing_price['mdate'] == '2018-07-24']

,mdate,coid,close_d
None,,,
0,2018-07-24,1101,45.5
16,2018-07-24,2330,241.0


## 2

7/25時2330收盤價240.5低於limit_price 240.6，於是成交。但1101股價還是高於43.5，沒有成交。

In [11]:
transactions

amount  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330       124   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101       725   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101       497   
                          Equity(1 [2330]) 2330        82   

                                                                         dt  \
date                      asset            symbol                             
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330   2018-07-25 13:30:00+08:00   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-31 13:30:00+08:00   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-02 13:30:00+08:00   
                          Equity(1 [2330]) 2330   2018-08-02 13:30:00+08:00   

                                                    price  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330    240.50   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101     39.35   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101     40.60   
                          Equity(1 [2330]) 2330    244.50   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330    a4a72ad3229541c9b655745f305c7b0d   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101    d4f04b538af140298289d1cc96620a1c   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101    6121b364417b471f92de68d5c246237f   
                          Equity(1 [2330]) 2330    00099a5ddcfb496db3b6aea457670fd3   

                                                  commission  
date                      asset            symbol             
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330         None  
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101         None  
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101         None  
                          Equity(1 [2330]) 2330         None

## 3

7/26時1101經過一次分割，比率 = 1/1.1，7/24訂單現在變成limit_price = 43.5 / 1.1 = 39.54，amount = 659 * 1.1 = 725。單子還繼續開著，到了7/31才以39.35成交。<br>
注意，購買量是用下訂日的總額、收盤價計算，成交時用成交日收盤價，所以購入的金額不是原訂的接近30000，也不會是成交日時的資產組合總額的30%。

註：amount的計算，函數應該是用捨去前的659.xxx，乘上1.1，再捨去。

In [12]:
performance['orders'][5]

[{'id': 'd4f04b538af140298289d1cc96620a1c',
  'dt': Timestamp('2018-07-31 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-24 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 725,
  'filled': 725,
  'commission': 40.65346875,
  'stop': None,
  'limit': 39.54,
  'stop_reached': False,
  'limit_reached': True,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>}]

## 4

接下來是8/1下單1101, 2330各20%，沒有stop/limit。用最新的portfolio value * 0.2 / 8/1收盤價，算出1101需要497股，2330需要82股，第二日成交。

#### 在8/1時總資產為102079，20%的金額為20415
#### 1101目標股數 = 20415 / 41.05 = 498股
#### 2330目標股數 = 20415 / 248 = 83股

In [13]:
performance['portfolio_value'][6]

102079.35018124999

In [14]:
closing_price[closing_price['mdate'] == '2018-08-01']

,mdate,coid,close_d
None,,,
6,2018-08-01,1101,41.05
22,2018-08-01,2330,248.00


#### 8/1交易訊號跑出

In [15]:
performance['orders'][6]

[{'id': '6121b364417b471f92de68d5c246237f',
  'dt': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 497,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.OPEN: 0>},
 {'id': '00099a5ddcfb496db3b6aea457670fd3',
  'dt': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 82,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2330]),
  'status': <ORDER_STATUS.OPEN: 0>}]

#### 8/2買入股票

In [16]:
performance['orders'][7]

[{'id': '6121b364417b471f92de68d5c246237f',
  'dt': Timestamp('2018-08-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 497,
  'filled': 497,
  'commission': 28.753935000000002,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>},
 {'id': '00099a5ddcfb496db3b6aea457670fd3',
  'dt': Timestamp('2018-08-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 82,
  'filled': 82,
  'commission': 28.569825,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2330]),
  'status': <ORDER_STATUS.FILLED: 1>}]

<span id="Order_target_percent"></span>
## Order_target_percent介紹
order_target的用法為限制住他的股票的總資產占比，如果持股價值超過設定的總資產權重，他就會賣出股票來達到目標權重。相對的，如果持股價值低於總資產權重，則買入股票。<br>
[Return to Menu](#menu)

## 投資1101、2330兩支股票

### 設置交易策略
#### def handle_data(context, data):

    在交易策略的第一個交易時間點（i 等於 0）時
    if context.i == 0:
        order_percent(symbol('1101'), 0.3, limit_price = 43.5)
        使用限價訂單購買股票 '1101'，量為投資組合當前價值的 30%，限價為 43.5
        order_percent(symbol('2330'), 0.3, limit_price = 240.6)
        使用限價訂單購買股票 '2330'，量為投資組合當前價值的 30%，限價為 240.6
    
    在交易策略的第一個交易時間點（i 等於 6）時
    if context.i == 6:
        
        order_percent(symbol('1101'), 0.2)
        購買股票 '1101'，量為投資組合當前價值的 20%
        order_percent(symbol('2330'), 0.2)
        購買股票 '2330'，量為投資組合當前價值的 20%
    
    記錄投資組合中所有資產的收盤價
    record(close=data.current(context.asset, 'close'))
    context.i += 1

In [17]:
def initialize(context):
    context.i = 0
    context.tickers = ['1101','2330']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order_target_percent(symbol('1101'), 0.3, limit_price = 43.5)
        order_target_percent(symbol('2330'), 0.3, limit_price = 240.6)
        
    if context.i == 6:
        order_target_percent(symbol('1101'), 0.2)
        order_target_percent(symbol('2330'), 0.2)

    record(close=data.current(context.asset, 'close'))
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e5
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')


In [19]:
closing_price = tejapi.get('TWN/APIPRCD',
                           coid=['1101','2330'], 
                           opts={'columns':['mdate','coid','close_d']}, 
                           mdate={'gte':start_dt,'lte':end_dt }, 
                           paginate=True)

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

## 回測結果

## 1

這邊7/25以及7/31的部分跟上面order_percent一樣

In [20]:
transactions

amount  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330       124   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101       725   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101      -227   
                          Equity(1 [2330]) 2330       -41   

                                                                         dt  \
date                      asset            symbol                             
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330   2018-07-25 13:30:00+08:00   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-31 13:30:00+08:00   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-02 13:30:00+08:00   
                          Equity(1 [2330]) 2330   2018-08-02 13:30:00+08:00   

                                                    price  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330    240.50   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101     39.35   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101     40.60   
                          Equity(1 [2330]) 2330    244.50   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330    147a0ea8bfff471d8fc51f31b0246e1c   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101    022aeb348f5c4ea88fa909a2b4f4fb29   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101    b05d6c0fda6f4fa890886708d55eb649   
                          Equity(1 [2330]) 2330    52fdc2dbfc214bec938bccbb83de0ff8   

                                                  commission  
date                      asset            symbol             
2018-07-25 00:00:00+08:00 Equity(1 [2330]) 2330         None  
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101         None  
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101         None  
                          Equity(1 [2330]) 2330         None

## 2

#### 在8/1時總資產為102079，20%的金額為20415
#### 1101目標股數 = 20415 / 41.05 = 498股，因此 725 - 498 = 227
     要將股數售出227筆，才能讓目標股數變成當下總資產的20%
#### 2330目標股數 = 20415 / 248 = 83股，因此 124 - 83 = 41
     要將股數售出41筆，才能讓目標股數變成當下總資產的20%

In [21]:
performance['portfolio_value'][6]

102079.35018124999

In [22]:
closing_price[closing_price['mdate'] == '2018-08-01']

,mdate,coid,close_d
None,,,
6,2018-08-01,1101,41.05
22,2018-08-01,2330,248.00


#### 8/1交易訊號跑出並下單

In [23]:
performance['orders'][6]

[{'id': 'b05d6c0fda6f4fa890886708d55eb649',
  'dt': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -227,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.OPEN: 0>},
 {'id': '52fdc2dbfc214bec938bccbb83de0ff8',
  'dt': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -41,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2330]),
  'status': <ORDER_STATUS.OPEN: 0>}]

#### 8/2賣出股票

In [24]:
performance['orders'][7]

[{'id': 'b05d6c0fda6f4fa890886708d55eb649',
  'dt': Timestamp('2018-08-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -227,
  'filled': -227,
  'commission': 13.133085000000001,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>},
 {'id': '52fdc2dbfc214bec938bccbb83de0ff8',
  'dt': Timestamp('2018-08-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-08-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -41,
  'filled': -41,
  'commission': 14.2849125,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2330]),
  'status': <ORDER_STATUS.FILLED: 1>}]